In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from pyvirtualdisplay import Display
from bs4 import BeautifulSoup

import time
import json
import os
import re

import spacy
import string

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ashis-
[nltk_data]     solomon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data Extraction

In [3]:
inputFile = pd.read_excel('./INPUT/Input.xlsx')

inputFile

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [4]:
len(inputFile)

114

In [5]:
article_text = ['' for i in range(len(inputFile))]

In [6]:
article_text[0]

''

In [7]:
skip_list = []
def data_extract():
    for i in range(len(inputFile)):
        try:
            url = inputFile.iloc[i]['URL']

            display = Display(visible=0, size=(800, 600))
            display.start()
            
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.implicitly_wait(10)
            wait = WebDriverWait(driver, 10)
            driver.get(url)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'article')))
            time.sleep(5)
            page = driver.page_source
            display.stop()
            driver.quit()

            soup = BeautifulSoup(page, 'html.parser')

            temp_article_title = soup.find_all('h1', class_='entry-title')[0].text
            temp_article_div = soup.find_all('div', class_='td-ss-main-content')
            temp_article_paras = []
            for d in temp_article_div:
                temp_article_paras.extend(d.find_all('p'))

            temp_article_paras = [x.text for x in temp_article_paras]
            temp_article_text = ' '.join(temp_article_paras)
            article_text[i] = temp_article_text
            
            # Saving the articles into <URL_ID>.txt files
            with open(f"./OUTPUT/Extracted_Articles/{inputFile.iloc[i]['URL_ID']}.txt","w") as f:
                f.writelines(temp_article_title + '\n\n')
                f.writelines('\n'.join(temp_article_paras))


            print(f'Done {i}')
        except:
            print(f'Skipped {i}')
            skip_list.append(i)

In [8]:
data_extract()

Done 0
Done 1
Done 2
Done 3
Done 4
Done 5
Done 6
Skipped 7
Done 8
Done 9
Done 10
Done 11
Done 12
Done 13
Skipped 14
Done 15
Done 16
Done 17
Done 18
Done 19
Skipped 20
Done 21
Done 22
Done 23
Done 24
Done 25
Done 26
Done 27
Done 28
Done 29
Done 30
Done 31
Done 32
Done 33
Done 34
Done 35
Done 36
Done 37
Done 38
Done 39
Done 40
Done 41
Done 42
Done 43
Done 44
Done 45
Done 46
Done 47
Done 48
Done 49
Done 50
Done 51
Done 52
Done 53
Skipped 54
Skipped 55
Done 56
Done 57
Done 58
Done 59
Done 60
Done 61
Done 62
Skipped 63
Done 64
Done 65
Done 66
Done 67
Done 68
Done 69
Skipped 70
Skipped 71
Done 72
Done 73
Done 74
Skipped 75
Done 76
Done 77
Done 78
Done 79
Done 80
Done 81
Done 82
Done 83
Done 84
Done 85
Done 86
Done 87
Done 88
Done 89
Done 90
Done 91
Done 92
Done 93
Done 94
Done 95
Done 96
Done 97
Done 98
Done 99
Done 100
Done 101
Done 102
Done 103
Done 104
Done 105
Done 106
Skipped 107
Done 108
Done 109
Done 110
Done 111
Done 112
Done 113


In [9]:
skip_list

[7, 14, 20, 54, 55, 63, 70, 71, 75, 107]

In [10]:
inputFile.loc[skip_list]

,URL_ID,URL
7,44,https://insights.blackcoffer.com/how-neural-ne...
14,51,https://insights.blackcoffer.com/future-of-wor...
20,57,https://insights.blackcoffer.com/covid-19-envi...
54,91,https://insights.blackcoffer.com/human-rights-...
55,92,https://insights.blackcoffer.com/how-voice-sea...
63,100,https://insights.blackcoffer.com/estimating-th...
70,107,https://insights.blackcoffer.com/how-covid-19-...
71,108,https://insights.blackcoffer.com/how-will-covi...
75,112,https://insights.blackcoffer.com/how-will-covi...
107,144,https://insights.blackcoffer.com/ensuring-grow...


### The articles are extracted into the article_text[ ] . But as we can notice some of the links are corrupted namely, the ones I have listed above. To resolve this issue I am going to drop the rows in the skip_list.

In [11]:
dataset = inputFile.copy()

In [12]:
dataset

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [13]:
dataset.drop(index=skip_list, inplace=True)

In [14]:
len(article_text)

114

In [15]:
list1 = article_text.copy()

In [16]:
list1 = [x for x in article_text if x != '']

In [17]:
len(list1)

104

In [18]:
article_text = list1

In [19]:
dataset.reset_index(inplace=True)

In [20]:
dataset

,index,URL_ID,URL
0,0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,1,38,https://insights.blackcoffer.com/what-if-the-c...
2,2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,3,40,https://insights.blackcoffer.com/will-machine-...
4,4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...,...
99,109,146,https://insights.blackcoffer.com/blockchain-fo...
100,110,147,https://insights.blackcoffer.com/the-future-of...
101,111,148,https://insights.blackcoffer.com/big-data-anal...
102,112,149,https://insights.blackcoffer.com/business-anal...


In [21]:
dataset.drop(['index'], axis=1, inplace=True)

In [22]:
dataset

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
99,146,https://insights.blackcoffer.com/blockchain-fo...
100,147,https://insights.blackcoffer.com/the-future-of...
101,148,https://insights.blackcoffer.com/big-data-anal...
102,149,https://insights.blackcoffer.com/business-anal...


In [23]:
article_text[0]

'Introduction “If anything kills over 10 million people in the next few decades, it will be a highly infectious virus rather than a war. Not missiles but microbes.” Bill Gates’s remarks at a TED conference in 2014, right after the world had avoided the Ebola outbreak. When the new, unprecedented, invisible virus hit us, it met an overwhelmed and unprepared healthcare system and oblivious population. This public health emergency demonstrated our lack of scientific consideration and underlined the alarming need for robust innovations in our health and medical facilities. For the past few years, artificial intelligence has proven to be of tangible potential in the healthcare sectors, clinical practices, translational medical and biomedical research. After the first case was detected in China on December 31st 2019, it was an AI program developed by BlueDot that alerted the world about the pandemic. It was quick to realise AI’s ability to analyse large chunks of data could help in detecting

In [24]:
len(article_text)

104

In [69]:
article_text_df = pd.DataFrame(article_text)

In [70]:
article_text_df

,0
0,Introduction “If anything kills over 10 millio...
1,"Human minds, a fascination in itself carrying ..."
2,Introduction AI is rapidly evolving in the emp...
3,“Anything that could give rise to smarter-than...
4,“Machine intelligence is the last invention th...
...,...
99,Reconciling with the financial realities of an...
100,An investment is a resource or thing procured ...
101,Quality and affordable healthcare is a vision ...
102,Analytics is a statistical scientific process ...


In [71]:
article_text_df.to_csv('./OUTPUT/article_text_saved.csv')

### The data extraction is complete. The articles are in the article_text [ ]. 

## Reading the Stop Words and creating the lists with Positive and Negative Words.

In [25]:
import os

directory = "./INPUT/StopWords" 

stop_words = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r", encoding="utf-8-sig") as file:
            words = [line.split()[0].lower() for line in file]
            stop_words.extend(words)

len(stop_words)

14107

In [26]:
with open('./INPUT/MasterDictionary/positive-words.txt') as f:
    positive_list = f.read().splitlines()
    
with open('./INPUT/MasterDictionary/negative-words.txt') as f:
    negative_list = f.read().splitlines()

In [27]:
def remove_stop_words_fromList(temp_list, stop_words):
    filtered_list = []
    for element in temp_list:
        if element not in stop_words:
            filtered_list.append(element)
    return filtered_list

positive_list = remove_stop_words_fromList(positive_list, stop_words)
negative_list = remove_stop_words_fromList(negative_list, stop_words)

In [28]:
# The stop words are removed from the lists.

In [29]:
stop_words

['ernst',
 'young',
 'deloitte',
 'touche',
 'kpmg',
 'pricewaterhousecoopers',
 'pricewaterhouse',
 'coopers',
 'united',
 'state',
 'north',
 'south',
 'east',
 'northeast',
 'northwest',
 'southeast',
 'southwest',
 'west',
 'ocean',
 'sea',
 'lake',
 'river',
 'creek',
 'gulf',
 'mountain',
 'street',
 'boulevard',
 'blvd',
 'parkway',
 'city',
 'county',
 'country',
 'pacific',
 'atlantic',
 'indian',
 'mediterranean',
 'commonwealth',
 'america',
 'american',
 'york',
 'chicago',
 'las',
 'vegas',
 'los',
 'angeles',
 'milwaukee',
 'sunnyvale',
 'fremont',
 'cincinnati',
 'philadelphia',
 'miami',
 'dallas',
 'fort',
 'boston',
 'houston',
 'washington',
 'atlanta',
 'detroit',
 'san',
 'fransico',
 'phoenix',
 'seattle',
 'diego',
 'minneapolis',
 'memphis',
 'denver',
 'st',
 'louis',
 'pittsburgh',
 'manhattan',
 'hollywood',
 'columbus',
 'indianapolis',
 'mumbai',
 'karachi',
 'ontario',
 'toronto',
 'cambridge',
 'delhi',
 'sao',
 'paulo',
 'shanghai',
 'moscow',
 'seoul',


In [30]:
positive_list

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [31]:
negative_list

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


# Extracting Sentiment Scores

### Defining the functions for sentiment_scores extraction.

In [32]:
# the string '\xa0' is not a part of the original text... so removing it
# also removing the punctuations

def preprocess_text_sentiment(text):
    nlp = spacy.load("en_core_web_sm")

    # Remove '\xa0' from the text
    text = text.replace('\xa0', '')

    doc = nlp(text)
    punctuations = string.punctuation
    tokens = [token for token in doc if not token.is_punct]
    clean_text = " ".join([token.text for token in tokens])

    return clean_text

In [33]:
def remove_stop_words_sentiment(text, stop_words):
    words = text.split()
    filtered_words = []

    for word in words:
        if word not in stop_words:
            filtered_words.append(word)
    filtered_text = " ".join(filtered_words)
    
    return filtered_text

In [34]:
# Tokenize the text

import nltk
nltk.download('punkt')

def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    return tokens

[nltk_data] Downloading package punkt to /home/ashis-
[nltk_data]     solomon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
def get_sentiment_scores(tokens):

    # Calculate positive and negative scores
    pos_score = sum([1 for token in tokens if token in positive_list])
    neg_score = sum([-1 for token in tokens if token in negative_list]) * -1
    
    # Calculate polarity score
    polarity_score = (pos_score - neg_score) / ((pos_score + neg_score) + 0.000001)
    
    # Calculate the subjectivity score
    total_words = len(tokens)
    subjectivity_score = (pos_score + neg_score) / (total_words + 0.000001)
    
    # Return scores as a dictionary
    scores = {
        'positive_score': pos_score,
        'negative_score': neg_score,
        'polarity_score': polarity_score,
        'subjectivity_score': subjectivity_score
    }
    return scores

In [36]:
sentiment_scores = ['' for _ in range(len(article_text))]

In [37]:
for i in range(len(article_text)):
    text = article_text[i].lower()
    text = preprocess_text_sentiment(text)
    text = remove_stop_words_sentiment(text, stop_words)
    tokens = tokenize_text(text)
    sentiment_scores[i] = get_sentiment_scores(tokens)

In [38]:
sentiment_scores

[{'positive_score': 64,
  'negative_score': 34,
  'polarity_score': 0.30612244585589343,
  'subjectivity_score': 0.10010214494371589},
 {'positive_score': 58,
  'negative_score': 37,
  'polarity_score': 0.22105262925207758,
  'subjectivity_score': 0.16183986343809223},
 {'positive_score': 64,
  'negative_score': 35,
  'polarity_score': 0.29292928997041123,
  'subjectivity_score': 0.11970979429297486},
 {'positive_score': 59,
  'negative_score': 27,
  'polarity_score': 0.372093018929151,
  'subjectivity_score': 0.14075286392675473},
 {'positive_score': 57,
  'negative_score': 25,
  'polarity_score': 0.3902438976799525,
  'subjectivity_score': 0.10314465395830862},
 {'positive_score': 43,
  'negative_score': 25,
  'polarity_score': 0.26470587846020766,
  'subjectivity_score': 0.12569316058097382},
 {'positive_score': 21,
  'negative_score': 11,
  'polarity_score': 0.3124999902343753,
  'subjectivity_score': 0.0975609753123141},
 {'positive_score': 32,
  'negative_score': 13,
  'polarity_

### The Sentiment Scores are extracted.

# Extracting Readability Variables : Part I

In [39]:
readability_vars_1 = ['' for _ in range(len(article_text))]

In [40]:
def calculate_readability(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())
    words = [word for word in words if word not in stop_words]

    # Calculate average sentence length
    avg_sentence_length = len(words) / len(sentences)

    # Calculate percentage of complex words
    complex_words = [word for word in words if len(word) > 2 and len(set(word)) >= 2]
    pct_complex_words = len(complex_words) / len(words)

    # Calculate Fog Index
    fog_index = 0.4 * (avg_sentence_length + pct_complex_words)

    # Return dictionary of results
    results = {'avg_sentence_length': avg_sentence_length,
               'pct_complex_words': pct_complex_words,
               'fog_index': fog_index}

    return results

In [41]:
for i in range(len(article_text)):
    readability_vars_1[i] = calculate_readability(article_text[i])

In [42]:
readability_vars_1

[{'avg_sentence_length': 15.413333333333334,
  'pct_complex_words': 0.8096885813148789,
  'fog_index': 6.489208765859286},
 {'avg_sentence_length': 9.632911392405063,
  'pct_complex_words': 0.7135348226018396,
  'fog_index': 4.1385784860027615},
 {'avg_sentence_length': 12.047619047619047,
  'pct_complex_words': 0.7905138339920948,
  'fog_index': 5.135253152644458},
 {'avg_sentence_length': 8.516483516483516,
  'pct_complex_words': 0.7664516129032258,
  'fog_index': 3.713174051754697},
 {'avg_sentence_length': 12.592105263157896,
  'pct_complex_words': 0.7659352142110762,
  'fog_index': 5.343216190947589},
 {'avg_sentence_length': 12.436363636363636,
  'pct_complex_words': 0.7426900584795322,
  'fog_index': 5.271621477937267},
 {'avg_sentence_length': 9.636363636363637,
  'pct_complex_words': 0.7405660377358491,
  'fog_index': 4.150771869639795},
 {'avg_sentence_length': 10.714285714285714,
  'pct_complex_words': 0.7733333333333333,
  'fog_index': 4.595047619047619},
 {'avg_sentence_le

### The Readability Variables : [ Avg. Sentence Length, Percentage of Complex Words, Fog Index ]
### are extracted.

# Extracting Readability Variables : Part II

In [43]:
# Defining Function

In [44]:
def average_words_per_sentence(text):
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)
    avg_words_per_sentence = len(words) / len(sentences)
    return avg_words_per_sentence

In [45]:
def complex_word_count(text):
    words = nltk.word_tokenize(text.lower())
    syllable_counts = syllable_count_per_word(text)
    complex_word_count = 0
    for i in range(len(words)):
        if syllable_counts[i] > 2:
            complex_word_count += 1
    return complex_word_count

In [46]:
def word_count(text):
    words = nltk.word_tokenize(text.lower())
    words = [word for word in words if word not in stopwords.words('english')]
    words = [word for word in words if word not in string.punctuation]
    return len(words)

In [47]:
def syllable_count_per_word(text):
    words = nltk.word_tokenize(text.lower())
    syllable_counts = []
    for word in words:
        vowel_count = 0
        prev_char = ''
        for char in word:
            if char in 'aeiouy' and prev_char not in 'aeiouy':
                vowel_count += 1
            prev_char = char
        if word.endswith('es') or word.endswith('ed'):
            vowel_count -= 1
        if vowel_count == 0:
            vowel_count = 1
        syllable_counts.append(vowel_count)
    return syllable_counts

def avg_syllable_count_per_word(text):
    words = nltk.word_tokenize(text.lower())
    syllable_counts = []
    for word in words:
        vowel_count = 0
        prev_char = ''
        for char in word:
            if char in 'aeiouy' and prev_char not in 'aeiouy':
                vowel_count += 1
            prev_char = char
        if word.endswith('es') or word.endswith('ed'):
            vowel_count -= 1
        if vowel_count == 0:
            vowel_count = 1
        syllable_counts.append(vowel_count)
    return sum(syllable_counts)/len(syllable_counts)

In [48]:
def count_personal_pronouns(text):
    pattern = r"\b(I|we|my|ours|us)\b"
    matches = re.findall(pattern, text)

    # Remove matches that are the country name "US"
    matches = [match for match in matches if match != 'US']
 
    count = len(matches)
    return count

In [49]:
def calculate_avg_word_length(text):
    words = re.findall(r'\b\w+\b', text)
    sum_char_lengths = sum(len(word) for word in words)
    total_words = len(words)
    avg_word_length = sum_char_lengths / total_words

    return avg_word_length

In [50]:
readability_vars_2 = ['' for _ in range(len(article_text))]

In [51]:
for i in range(len(article_text)):
    readability_vars_2[i] = {
        'average_words_per_sentence': average_words_per_sentence(article_text[i]),
        'complex_word_count': complex_word_count(article_text[i]),
        'word_count': word_count(article_text[i]),
        'avg_syllable_count_per_word': avg_syllable_count_per_word(article_text[i]),
        'count_personal_pronouns': count_personal_pronouns(article_text[i]),
        'calculate_avg_word_length': calculate_avg_word_length(article_text[i])
    }

In [52]:
readability_vars_2

[{'average_words_per_sentence': 26.546666666666667,
  'complex_word_count': 386,
  'word_count': 1148,
  'avg_syllable_count_per_word': 1.707182320441989,
  'count_personal_pronouns': 1,
  'calculate_avg_word_length': 5.553097345132743},
 {'average_words_per_sentence': 20.620253164556964,
  'complex_word_count': 184,
  'word_count': 768,
  'avg_syllable_count_per_word': 1.452088452088452,
  'count_personal_pronouns': 6,
  'calculate_avg_word_length': 4.716376306620209},
 {'average_words_per_sentence': 22.607142857142858,
  'complex_word_count': 348,
  'word_count': 996,
  'avg_syllable_count_per_word': 1.6413902053712481,
  'count_personal_pronouns': 2,
  'calculate_avg_word_length': 5.35314891112419},
 {'average_words_per_sentence': 19.384615384615383,
  'complex_word_count': 228,
  'word_count': 882,
  'avg_syllable_count_per_word': 1.5073696145124718,
  'count_personal_pronouns': 17,
  'calculate_avg_word_length': 4.777917189460477},
 {'average_words_per_sentence': 25.31578947368421

### The remaining Readability Variables are extracted.

In [53]:
sentiment_scores

[{'positive_score': 64,
  'negative_score': 34,
  'polarity_score': 0.30612244585589343,
  'subjectivity_score': 0.10010214494371589},
 {'positive_score': 58,
  'negative_score': 37,
  'polarity_score': 0.22105262925207758,
  'subjectivity_score': 0.16183986343809223},
 {'positive_score': 64,
  'negative_score': 35,
  'polarity_score': 0.29292928997041123,
  'subjectivity_score': 0.11970979429297486},
 {'positive_score': 59,
  'negative_score': 27,
  'polarity_score': 0.372093018929151,
  'subjectivity_score': 0.14075286392675473},
 {'positive_score': 57,
  'negative_score': 25,
  'polarity_score': 0.3902438976799525,
  'subjectivity_score': 0.10314465395830862},
 {'positive_score': 43,
  'negative_score': 25,
  'polarity_score': 0.26470587846020766,
  'subjectivity_score': 0.12569316058097382},
 {'positive_score': 21,
  'negative_score': 11,
  'polarity_score': 0.3124999902343753,
  'subjectivity_score': 0.0975609753123141},
 {'positive_score': 32,
  'negative_score': 13,
  'polarity_

In [54]:
readability_vars_1

[{'avg_sentence_length': 15.413333333333334,
  'pct_complex_words': 0.8096885813148789,
  'fog_index': 6.489208765859286},
 {'avg_sentence_length': 9.632911392405063,
  'pct_complex_words': 0.7135348226018396,
  'fog_index': 4.1385784860027615},
 {'avg_sentence_length': 12.047619047619047,
  'pct_complex_words': 0.7905138339920948,
  'fog_index': 5.135253152644458},
 {'avg_sentence_length': 8.516483516483516,
  'pct_complex_words': 0.7664516129032258,
  'fog_index': 3.713174051754697},
 {'avg_sentence_length': 12.592105263157896,
  'pct_complex_words': 0.7659352142110762,
  'fog_index': 5.343216190947589},
 {'avg_sentence_length': 12.436363636363636,
  'pct_complex_words': 0.7426900584795322,
  'fog_index': 5.271621477937267},
 {'avg_sentence_length': 9.636363636363637,
  'pct_complex_words': 0.7405660377358491,
  'fog_index': 4.150771869639795},
 {'avg_sentence_length': 10.714285714285714,
  'pct_complex_words': 0.7733333333333333,
  'fog_index': 4.595047619047619},
 {'avg_sentence_le

In [55]:
readability_vars_2

[{'average_words_per_sentence': 26.546666666666667,
  'complex_word_count': 386,
  'word_count': 1148,
  'avg_syllable_count_per_word': 1.707182320441989,
  'count_personal_pronouns': 1,
  'calculate_avg_word_length': 5.553097345132743},
 {'average_words_per_sentence': 20.620253164556964,
  'complex_word_count': 184,
  'word_count': 768,
  'avg_syllable_count_per_word': 1.452088452088452,
  'count_personal_pronouns': 6,
  'calculate_avg_word_length': 4.716376306620209},
 {'average_words_per_sentence': 22.607142857142858,
  'complex_word_count': 348,
  'word_count': 996,
  'avg_syllable_count_per_word': 1.6413902053712481,
  'count_personal_pronouns': 2,
  'calculate_avg_word_length': 5.35314891112419},
 {'average_words_per_sentence': 19.384615384615383,
  'complex_word_count': 228,
  'word_count': 882,
  'avg_syllable_count_per_word': 1.5073696145124718,
  'count_personal_pronouns': 17,
  'calculate_avg_word_length': 4.777917189460477},
 {'average_words_per_sentence': 25.31578947368421

## Merging the lists into a dataframe.

In [56]:
merged_data = []

for i in range(len(sentiment_scores)):
    data = {}
    data.update(sentiment_scores[i])
    data.update(readability_vars_1[i])
    data.update(readability_vars_2[i])
    merged_data.append(data)

In [57]:
merged_data

[{'positive_score': 64,
  'negative_score': 34,
  'polarity_score': 0.30612244585589343,
  'subjectivity_score': 0.10010214494371589,
  'avg_sentence_length': 15.413333333333334,
  'pct_complex_words': 0.8096885813148789,
  'fog_index': 6.489208765859286,
  'average_words_per_sentence': 26.546666666666667,
  'complex_word_count': 386,
  'word_count': 1148,
  'avg_syllable_count_per_word': 1.707182320441989,
  'count_personal_pronouns': 1,
  'calculate_avg_word_length': 5.553097345132743},
 {'positive_score': 58,
  'negative_score': 37,
  'polarity_score': 0.22105262925207758,
  'subjectivity_score': 0.16183986343809223,
  'avg_sentence_length': 9.632911392405063,
  'pct_complex_words': 0.7135348226018396,
  'fog_index': 4.1385784860027615,
  'average_words_per_sentence': 20.620253164556964,
  'complex_word_count': 184,
  'word_count': 768,
  'avg_syllable_count_per_word': 1.452088452088452,
  'count_personal_pronouns': 6,
  'calculate_avg_word_length': 4.716376306620209},
 {'positive_s

In [58]:
df = pd.DataFrame(merged_data)

In [59]:
df

,positive_score,negative_score,polarity_score,subjectivity_score,avg_sentence_length,pct_complex_words,fog_index,average_words_per_sentence,complex_word_count,word_count,avg_syllable_count_per_word,count_personal_pronouns,calculate_avg_word_length
0,64,34,0.306122,0.100102,15.413333,0.809689,6.489209,26.546667,386,1148,1.707182,1,5.553097
1,58,37,0.221053,0.161840,9.632911,0.713535,4.138578,20.620253,184,768,1.452088,6,4.716376
2,64,35,0.292929,0.119710,12.047619,0.790514,5.135253,22.607143,348,996,1.641390,2,5.353149
3,59,27,0.372093,0.140753,8.516484,0.766452,3.713174,19.384615,228,882,1.507370,17,4.777917
4,57,25,0.390244,0.103145,12.592105,0.765935,5.343216,25.315789,261,980,1.532744,12,4.938940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,21,27,-0.125000,0.111369,10.416667,0.816000,4.493067,20.416667,165,531,1.617347,9,5.141928
100,31,11,0.476190,0.078652,13.000000,0.784929,5.513972,24.918367,191,651,1.581148,1,5.052867
101,26,45,-0.267606,0.122203,10.523077,0.812865,4.534377,19.307692,228,672,1.604781,2,4.953712
102,27,5,0.687500,0.115523,13.320000,0.825826,5.658330,24.640000,126,326,1.659091,0,5.545455


In [60]:
df.columns

Index(['positive_score', 'negative_score', 'polarity_score',
       'subjectivity_score', 'avg_sentence_length', 'pct_complex_words',
       'fog_index', 'average_words_per_sentence', 'complex_word_count',
       'word_count', 'avg_syllable_count_per_word', 'count_personal_pronouns',
       'calculate_avg_word_length'],
      dtype='object')

In [61]:
col_list = ['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 
            'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 
            'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
            'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']

In [62]:
len(col_list)

13

In [63]:
df = df.rename(columns=dict(zip(df.columns, col_list)))

In [64]:
df

,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,64,34,0.306122,0.100102,15.413333,0.809689,6.489209,26.546667,386,1148,1.707182,1,5.553097
1,58,37,0.221053,0.161840,9.632911,0.713535,4.138578,20.620253,184,768,1.452088,6,4.716376
2,64,35,0.292929,0.119710,12.047619,0.790514,5.135253,22.607143,348,996,1.641390,2,5.353149
3,59,27,0.372093,0.140753,8.516484,0.766452,3.713174,19.384615,228,882,1.507370,17,4.777917
4,57,25,0.390244,0.103145,12.592105,0.765935,5.343216,25.315789,261,980,1.532744,12,4.938940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,21,27,-0.125000,0.111369,10.416667,0.816000,4.493067,20.416667,165,531,1.617347,9,5.141928
100,31,11,0.476190,0.078652,13.000000,0.784929,5.513972,24.918367,191,651,1.581148,1,5.052867
101,26,45,-0.267606,0.122203,10.523077,0.812865,4.534377,19.307692,228,672,1.604781,2,4.953712
102,27,5,0.687500,0.115523,13.320000,0.825826,5.658330,24.640000,126,326,1.659091,0,5.545455


In [65]:
dataset

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
99,146,https://insights.blackcoffer.com/blockchain-fo...
100,147,https://insights.blackcoffer.com/the-future-of...
101,148,https://insights.blackcoffer.com/big-data-anal...
102,149,https://insights.blackcoffer.com/business-anal...


In [66]:
merged_df = pd.concat([dataset, df], axis=1)

In [67]:
merged_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,64,34,0.306122,0.100102,15.413333,0.809689,6.489209,26.546667,386,1148,1.707182,1,5.553097
1,38,https://insights.blackcoffer.com/what-if-the-c...,58,37,0.221053,0.161840,9.632911,0.713535,4.138578,20.620253,184,768,1.452088,6,4.716376
2,39,https://insights.blackcoffer.com/what-jobs-wil...,64,35,0.292929,0.119710,12.047619,0.790514,5.135253,22.607143,348,996,1.641390,2,5.353149
3,40,https://insights.blackcoffer.com/will-machine-...,59,27,0.372093,0.140753,8.516484,0.766452,3.713174,19.384615,228,882,1.507370,17,4.777917
4,41,https://insights.blackcoffer.com/will-ai-repla...,57,25,0.390244,0.103145,12.592105,0.765935,5.343216,25.315789,261,980,1.532744,12,4.938940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,146,https://insights.blackcoffer.com/blockchain-fo...,21,27,-0.125000,0.111369,10.416667,0.816000,4.493067,20.416667,165,531,1.617347,9,5.141928
100,147,https://insights.blackcoffer.com/the-future-of...,31,11,0.476190,0.078652,13.000000,0.784929,5.513972,24.918367,191,651,1.581148,1,5.052867
101,148,https://insights.blackcoffer.com/big-data-anal...,26,45,-0.267606,0.122203,10.523077,0.812865,4.534377,19.307692,228,672,1.604781,2,4.953712
102,149,https://insights.blackcoffer.com/business-anal...,27,5,0.687500,0.115523,13.320000,0.825826,5.658330,24.640000,126,326,1.659091,0,5.545455


In [72]:
merged_df.to_csv('./OUTPUT/submission_output.csv')

# The Output is obtained.

### Saved as 'submission_output.csv' in ./OUTPUT